## Importing the data

In [ ]:
!pip install opendatasets --upgrade --quiet

In [ ]:
import opendatasets as od

In [ ]:
dataset_url = 'https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000'

In [ ]:
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: vankapgrover
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000


100%|██████████| 5.20G/5.20G [04:21<00:00, 21.4MB/s]


## Installing dependencies

In [ ]:
# !pip uninstall tensorflow
# !pip install tensorflow==2.8.0

Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Would remove:
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.17.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-2.17.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 107.1 MB/s eta 0:00:00
   ━

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.8.0


In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from keras import backend as K
from keras.layers import Layer,InputSpec
import keras.layers as kl
from glob import glob
from sklearn.metrics import roc_curve, auc
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_auc_score
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from  matplotlib import pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate,Dense, Conv2D, MaxPooling2D, Flatten,Input,Activation,add,AveragePooling2D,BatchNormalization,Dropout
%matplotlib inline
import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from tensorflow.python.platform import build_info as tf_build_info
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

## Preprocessing Data

In [ ]:
data_df = pd.read_csv('/content/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')
data_df

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face


In [ ]:
train_dir = os.path.join('skin-cancer-mnist-ham10000', 'train_dir')
test_dir = os.path.join('skin-cancer-mnist-ham10000', 'test_dir')

In [ ]:
count_df = data_df.groupby('lesion_id').count()
count_df

,image_id,dx,dx_type,age,sex,localization
lesion_id,,,,,,
HAM_0000000,2,2,2,2,2,2
HAM_0000001,1,1,1,1,1,1
HAM_0000002,3,3,3,3,3,3
HAM_0000003,1,1,1,1,1,1
HAM_0000004,1,1,1,1,1,1
...,...,...,...,...,...,...
HAM_0007624,1,1,1,1,1,1
HAM_0007625,2,2,2,2,2,2
HAM_0007626,1,1,1,1,1,1


In [ ]:
count_df = count_df[count_df['dx'] == 1]
count_df.reset_index(inplace=True)

In [ ]:
def labelDuplicates(x):
    unique = set(count_df['lesion_id'])
    if x in unique:
        return 'no_duplicates'
    else:
        return 'has_duplicates'

In [ ]:
data_df['is_duplicate'] = data_df['lesion_id'].apply(labelDuplicates)
data_df.sample(5)

,lesion_id,image_id,dx,dx_type,age,sex,localization,is_duplicate
4863,HAM_0000008,ISIC_0029868,nv,follow_up,65.0,male,trunk,no_duplicates
4420,HAM_0006656,ISIC_0028192,nv,follow_up,45.0,male,trunk,no_duplicates
5147,HAM_0004771,ISIC_0024622,nv,follow_up,55.0,male,trunk,no_duplicates
8157,HAM_0001037,ISIC_0031726,nv,histo,35.0,male,foot,has_duplicates
5646,HAM_0002594,ISIC_0031047,nv,follow_up,45.0,female,upper extremity,no_duplicates


In [ ]:
count_df = data_df[data_df['is_duplicate'] == 'no_duplicates']

In [ ]:
train_df, test_df = train_test_split(count_df, test_size=0.15, stratify=count_df['dx'])

In [ ]:
def identify_train_test(x):
    test_data = set(test_df['image_id'])
    if str(x) in test_data:
        return 'test'
    else:
        return 'train'

data_df['train_test_split'] = data_df['image_id'].apply(identify_train_test)
train_df = data_df[data_df['train_test_split'] == 'train']

In [ ]:
train_df.sample(5)

,lesion_id,image_id,dx,dx_type,age,sex,localization,is_duplicate,train_test_split
2840,HAM_0005371,ISIC_0027046,bcc,histo,65.0,female,face,has_duplicates,train
4861,HAM_0001746,ISIC_0027296,nv,follow_up,65.0,female,lower extremity,no_duplicates,train
2051,HAM_0002752,ISIC_0025748,mel,histo,70.0,male,lower extremity,no_duplicates,train
175,HAM_0006801,ISIC_0032688,bkl,histo,60.0,female,lower extremity,has_duplicates,train
6608,HAM_0006530,ISIC_0029686,nv,follow_up,50.0,male,abdomen,no_duplicates,train


In [ ]:
test_df.sample(5)

,lesion_id,image_id,dx,dx_type,age,sex,localization,is_duplicate
8661,HAM_0000338,ISIC_0028631,nv,histo,30.0,male,back,no_duplicates
3262,HAM_0000327,ISIC_0026682,nv,follow_up,50.0,female,trunk,no_duplicates
5955,HAM_0007344,ISIC_0029461,nv,follow_up,45.0,male,foot,no_duplicates
4892,HAM_0001165,ISIC_0030199,nv,follow_up,40.0,male,upper extremity,no_duplicates
6370,HAM_0003440,ISIC_0026943,nv,follow_up,30.0,female,lower extremity,no_duplicates


In [ ]:
train_list = list(train_df['image_id'])
test_list = list(test_df['image_id'])

In [ ]:
print(f'Number of images in train set: {len(train_list)}')
print(f'Number of images in test set: {len(test_list)}')

Number of images in train set: 9187
Number of images in test set: 828


In [ ]:
data_df.set_index('image_id', inplace=True)

In [ ]:
os.mkdir(train_dir)
os.mkdir(test_dir)

In [ ]:
target_names = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [ ]:
for i in target_names:
    dir_1 = os.path.join(train_dir, i)
    dir_2 = os.path.join(test_dir, i)
    os.mkdir(dir_1)
    os.mkdir(dir_2)

In [ ]:
for image in train_list:
    file_name = image + '.jpg'
    label = data_df.loc[image, 'dx']

    src = os.path.join('skin-cancer-mnist-ham10000', 'HAM10000_images_part_1', file_name)
    if os.path.exists(src):
        target = os.path.join(train_dir, label, file_name)
        shutil.copyfile(src, target)
    else:
        src = os.path.join('skin-cancer-mnist-ham10000', 'HAM10000_images_part_2', file_name)
        if os.path.exists(src):
            target = os.path.join(train_dir, label, file_name)
            shutil.copyfile(src, target)
        else:
            print(f"Image {file_name} not found in either folder.")

In [ ]:
for image in test_list:
    file_name = image + '.jpg'
    label = data_df.loc[image, 'dx']

    src = os.path.join('skin-cancer-mnist-ham10000', 'HAM10000_images_part_1', file_name)
    if os.path.exists(src):
        target = os.path.join(test_dir, label, file_name)
        shutil.copyfile(src, target)
    else:
        src = os.path.join('skin-cancer-mnist-ham10000', 'HAM10000_images_part_2', file_name)
        if os.path.exists(src):
            target = os.path.join(test_dir, label, file_name)
            shutil.copyfile(src, target)
        else:
            print(f"Image {file_name} not found in either folder.")

In [ ]:
# Augmenting images and storing them in temporary directories
for img_class in target_names:
    #creating temporary directories
    # creating a base directory
    aug_dir = 'aug_dir'
    os.mkdir(aug_dir)
    # creating a subdirectory inside the base directory for images of the same class
    img_dir = os.path.join(aug_dir, 'img_dir')
    os.mkdir(img_dir)

    img_list = os.listdir('skin-cancer-mnist-ham10000/train_dir/' + img_class)

    # Copy images from the class train dir to the img_dir
    for file_name in img_list:

        # path of source image in training directory
        source = os.path.join('skin-cancer-mnist-ham10000/train_dir/' + img_class, file_name)

        # creating a target directory to send images
        target = os.path.join(img_dir, file_name)

        # copying the image from the source to target file
        shutil.copyfile(source, target)

    # Temporary augumented dataset directory.
    source_path = aug_dir

    # Augmented images will be saved to training directory
    save_path = 'skin-cancer-mnist-ham10000/train_dir/' + img_class

    # Creating Image Data Generator to augment images
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=180,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
    )

    batch_size = 50

    aug_datagen = datagen.flow_from_directory(
        source_path,
        save_to_dir=save_path,
        save_format='jpg',
        target_size=(299, 299),
        batch_size=batch_size
    )

    # Generate the augmented images
    aug_images = 8000

    num_files = len(os.listdir(img_dir))
    num_batches = int(np.ceil((aug_images - num_files) / batch_size))

    # creating 8000 augmented images per class
    for i in range(0, num_batches):
        images, labels = next(aug_datagen)

    # delete temporary directory
    shutil.rmtree('aug_dir')


In [ ]:
train_path = 'skin-cancer-mnist-ham10000/train_dir'
test_path = 'skin-cancer-mnist-ham10000/test_dir'
batch_size = 16

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=180,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
    )

image_size = 299
print(f'Image size: {image_size} x {image_size}')
print('\nTrain Batches')
train_batches = datagen.flow_from_directory(
    directory=train_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)
print('\nTest Batches')
test_batches = datagen.flow_from_directory(
    directory=test_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Image size: 299 x 299

Train Batches
Found 9187 images belonging to 7 classes.

Test Batches
Found 828 images belonging to 7 classes.


## Building Model Architecture

### Soft Attention Layer

In [ ]:
from keras import backend as K
from keras.layers import Layer, InputSpec
import keras.layers as kl
import tensorflow as tf

In [ ]:
class SoftAttention(Layer):
    def __init__(self,ch,m,concat_with_x=False,aggregate=False,**kwargs):
        self.channels=int(ch)
        self.multiheads = m
        self.aggregate_channels = aggregate
        self.concat_input_with_scaled = concat_with_x


        super(SoftAttention,self).__init__(**kwargs)

    def build(self,input_shape):

        self.i_shape = input_shape

        kernel_shape_conv3d = (self.channels, 3, 3) + (1, self.multiheads) # DHWC

        self.out_attention_maps_shape = input_shape[0:1]+(self.multiheads,)+input_shape[1:-1]

        if self.aggregate_channels==False:

            self.out_features_shape = input_shape[:-1]+(input_shape[-1]+(input_shape[-1]*self.multiheads),)
        else:
            if self.concat_input_with_scaled:
                self.out_features_shape = input_shape[:-1]+(input_shape[-1]*2,)
            else:
                self.out_features_shape = input_shape


        self.kernel_conv3d = self.add_weight(shape=kernel_shape_conv3d,
                                        initializer='he_uniform',
                                        name='kernel_conv3d')
        self.bias_conv3d = self.add_weight(shape=(self.multiheads,),
                                      initializer='zeros',
                                      name='bias_conv3d')

        super(SoftAttention, self).build(input_shape)

    def call(self, x):

        exp_x = K.expand_dims(x,axis=-1)

        c3d = K.conv3d(exp_x,
                     kernel=self.kernel_conv3d,
                     strides=(1,1,self.i_shape[-1]), padding='same', data_format='channels_last')
        conv3d = K.bias_add(c3d,
                        self.bias_conv3d)
        conv3d = kl.Activation('relu')(conv3d)

        conv3d = K.permute_dimensions(conv3d,pattern=(0,4,1,2,3))


        conv3d = K.squeeze(conv3d, axis=-1)
        conv3d = K.reshape(conv3d,shape=(-1, self.multiheads ,self.i_shape[1]*self.i_shape[2]))

        softmax_alpha = K.softmax(conv3d, axis=-1)
        softmax_alpha = kl.Reshape(target_shape=(self.multiheads, self.i_shape[1],self.i_shape[2]))(softmax_alpha)


        if self.aggregate_channels==False:
            exp_softmax_alpha = K.expand_dims(softmax_alpha, axis=-1)
            exp_softmax_alpha = K.permute_dimensions(exp_softmax_alpha,pattern=(0,2,3,1,4))

            x_exp = K.expand_dims(x,axis=-2)

            u = kl.Multiply()([exp_softmax_alpha, x_exp])

            u = kl.Reshape(target_shape=(self.i_shape[1],self.i_shape[2],u.shape[-1]*u.shape[-2]))(u)

        else:
            exp_softmax_alpha = K.permute_dimensions(softmax_alpha,pattern=(0,2,3,1))

            exp_softmax_alpha = K.sum(exp_softmax_alpha,axis=-1)

            exp_softmax_alpha = K.expand_dims(exp_softmax_alpha, axis=-1)

            u = kl.Multiply()([exp_softmax_alpha, x])

        if self.concat_input_with_scaled:
            o = kl.Concatenate(axis=-1)([u,x])
        else:
            o = u

        return [o, softmax_alpha]

    def compute_output_shape(self, input_shape):
        return [self.out_features_shape, self.out_attention_maps_shape]


    def get_config(self):
        return super(SoftAttention,self).get_config()


In [ ]:
irv2 = tf.keras.applications.InceptionResNetV2(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classifier_activation="softmax",
)

# Excluding the last 28 layers of the model.
conv = irv2.layers[-28].output

In [ ]:
# Soft Attention
attention_layer,map2 = SoftAttention(aggregate=True,m=16,concat_with_x=False,ch=int(conv.shape[-1]),name='soft_attention')(conv)
attention_layer=(MaxPooling2D(pool_size=(2, 2),padding="same")(attention_layer))
conv=(MaxPooling2D(pool_size=(2, 2),padding="same")(conv))

conv = concatenate([conv,attention_layer])
conv  = Activation('relu')(conv)
conv = Dropout(0.5)(conv)

In [ ]:
output = Flatten()(conv)
output = Dense(7, activation='softmax')(output)
model = Model(inputs=irv2.input, outputs=output)

In [ ]:
model.summary()

In [ ]:
optimizer1 = tf.keras.optimizers.Adam(learning_rate=0.01, epsilon=0.1)
model.compile(
    optimizer=optimizer1,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
class_weights = {
    0: 1.0,  # akiec
    1: 1.0,  # bcc
    2: 1.0,  # bkl
    3: 1.0,  # df
    4: 5.0,  # mel
    5: 1.0,  # nv
    6: 1.0,  # vasc
}

checkpoint=  ModelCheckpoint(filepath ='IRV2+SA.hdf5',monitor='val_accuracy',save_best_only=True,save_weights_only=True)

In [ ]:
Earlystop = EarlyStopping(monitor='val_loss', mode='min',patience=30, min_delta=0.001)
history = model.fit(train_batches,
                    steps_per_epoch=int(len(train_df)/10),
                    epochs=150,
                    verbose=0,
                    validation_data=test_batches,validation_steps=len(test_df)/batch_size,callbacks=[checkpoint,Earlystop],class_weight=class_weights)

In [ ]:
from tensorflow.keras import models
model.load_weights('IRV2+SA.weights.h5')

In [ ]:
#geting predictions on test dataset
y_pred = np.argmax(predictions, axis=1)
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
#getting the true labels per image
y_true = test_batches.classes
#getting the predicted labels per image
y_prob=predictions
from tensorflow.keras.utils import to_categorical
y_test = to_categorical(y_true)

# Creating classification report
report = classification_report(y_true, y_pred, target_names=targetnames)

print("\nClassification Report:")
print(report)

In [ ]:
model.save('skin-model')